# Context
This notebook drives the training process for different models.

In [1]:
# Set project's environment variables
import os
import sys
from dotenv import load_dotenv
load_dotenv(dotenv_path="../project.env")
sys.path.append(os.environ["PYTHONPATH"])

import pickle

In [2]:
# Import project-wide and PH2 specific variables and functions
import superheader as sup
import TRAINheader as train



Chosen class grouping: ten-classes


Directory /Users/diego/Desktop/iteso/TOG/ exists. Continuing with execution
Directory /Users/diego/Desktop/iteso/TOG/data exists. Continuing with execution
Directory /Users/diego/Desktop/iteso/TOG/src exists. Continuing with execution
Directory /Users/diego/Desktop/iteso/TOG/bin exists. Continuing with execution
Directory /Users/diego/Desktop/iteso/TOG/media exists. Continuing with execution
Directory /Users/diego/Desktop/iteso/TOG/scores exists. Continuing with execution
Directory /Users/diego/Desktop/iteso/TOG/data/PH3/ten-classes exists. Continuing with execution
Directory /Users/diego/Desktop/iteso/TOG/bin/gen/TRAIN exists. Continuing with execution
Directory /Users/diego/Desktop/iteso/TOG/data/PH1/ten-classes exists. Continuing with execution
Directory /Users/diego/Desktop/iteso/TOG/data/PH2/ten-classes exists. Continuing with execution
Directory /Users/diego/Desktop/iteso/TOG/data/PH3/ten-classes exists. Continuing with execution
device: mps

# Model training

In [3]:
TRAIN_classes = 'ten-classes'

## KNN

### Active Hand prediction

In [4]:
train.arch.find_best(sup.DATA_AH_PF, sup.active_hand_col, TRAIN_classes, sup.TRAIN_KNN_CODE)

updating best... 0.9315535929345644
	{'PH2': True, 'PH3': True, 'reducer': 'pca', 'kernel': '', 'n': 1, 'data_unit': 'AHpf', 'label_col': 'active_hand', 'class_list': 'ten-classes'}
	{'arch': 'KNN', 'k': 1}
Directory /Users/diego/Desktop/iteso/TOG/bin/gen/TRAIN/ten-classes/KNN/AHpf does not exist. Creating it and continuing withexecution
updating best... 0.9504215174628663
	{'PH2': True, 'PH3': True, 'reducer': 'pca', 'kernel': '', 'n': 1, 'data_unit': 'AHpf', 'label_col': 'active_hand', 'class_list': 'ten-classes'}
	{'arch': 'KNN', 'k': 3}
Directory /Users/diego/Desktop/iteso/TOG/bin/gen/TRAIN/ten-classes/KNN/AHpf exists. Continuing with execution
updating best... 0.9520272982737856
	{'PH2': True, 'PH3': True, 'reducer': 'pca', 'kernel': '', 'n': 1, 'data_unit': 'AHpf', 'label_col': 'active_hand', 'class_list': 'ten-classes'}
	{'arch': 'KNN', 'k': 5}
Directory /Users/diego/Desktop/iteso/TOG/bin/gen/TRAIN/ten-classes/KNN/AHpf exists. Continuing with execution
updating best... 0.9558410

### Sign prediction

In [5]:
train.arch.find_best(sup.DATA_S_PF, sup.class_numeric_column, TRAIN_classes, sup.TRAIN_KNN_CODE)

updating best... 0.16171224732461356
	{'PH2': True, 'PH3': True, 'reducer': 'pca', 'kernel': '', 'n': 1, 'data_unit': 'Spf', 'label_col': 'class_numeric', 'class_list': 'ten-classes'}
	{'arch': 'KNN', 'k': 1}
Directory /Users/diego/Desktop/iteso/TOG/bin/gen/TRAIN/ten-classes/KNN/Spf does not exist. Creating it and continuing withexecution
updating best... 0.1636940150614348
	{'PH2': True, 'PH3': True, 'reducer': 'pca', 'kernel': '', 'n': 1, 'data_unit': 'Spf', 'label_col': 'class_numeric', 'class_list': 'ten-classes'}
	{'arch': 'KNN', 'k': 3}
Directory /Users/diego/Desktop/iteso/TOG/bin/gen/TRAIN/ten-classes/KNN/Spf exists. Continuing with execution
updating best... 0.16646848989298454
	{'PH2': True, 'PH3': True, 'reducer': 'pca', 'kernel': '', 'n': 1, 'data_unit': 'Spf', 'label_col': 'class_numeric', 'class_list': 'ten-classes'}
	{'arch': 'KNN', 'k': 4}
Directory /Users/diego/Desktop/iteso/TOG/bin/gen/TRAIN/ten-classes/KNN/Spf exists. Continuing with execution
updating best... 0.17479

In [6]:
train.arch.find_best(sup.DATA_S_PV, sup.class_numeric_column, TRAIN_classes, sup.TRAIN_KNN_CODE)

updating best... 0.2890995260663507
	{'PH2': True, 'PH3': True, 'reducer': 'pca', 'kernel': '', 'n': 1, 'data_unit': 'Spv', 'label_col': 'class_numeric', 'class_list': 'ten-classes'}
	{'arch': 'KNN', 'k': 1}
Directory /Users/diego/Desktop/iteso/TOG/bin/gen/TRAIN/ten-classes/KNN/Spv does not exist. Creating it and continuing withexecution
updating best... 0.3127962085308057
	{'PH2': True, 'PH3': True, 'reducer': 'umap', 'kernel': '', 'n': 1, 'data_unit': 'Spv', 'label_col': 'class_numeric', 'class_list': 'ten-classes'}
	{'arch': 'KNN', 'k': 1}
Directory /Users/diego/Desktop/iteso/TOG/bin/gen/TRAIN/ten-classes/KNN/Spv exists. Continuing with execution
updating best... 0.44549763033175355
	{'PH2': True, 'PH3': True, 'reducer': 'pca', 'kernel': '', 'n': 2, 'data_unit': 'Spv', 'label_col': 'class_numeric', 'class_list': 'ten-classes'}
	{'arch': 'KNN', 'k': 1}
Directory /Users/diego/Desktop/iteso/TOG/bin/gen/TRAIN/ten-classes/KNN/Spv exists. Continuing with execution
updating best... 0.46445

## BERT

In [7]:
train.arch.find_best(sup.DATA_S_PV, sup.class_numeric_column, TRAIN_classes, sup.TRAIN_BERT_CODE)

(1051, 12) {'PH2': True, 'PH3': True, 'reducer': 'pca', 'kernel': '', 'n': 1, 'data_unit': 'Spv', 'label_col': 'class_numeric', 'class_list': 'ten-classes', 'batch_size': 1024, 'input_dim': 1}
updating best... 0.10426540284360189
	{'PH2': True, 'PH3': True, 'reducer': 'pca', 'kernel': '', 'n': 1, 'data_unit': 'Spv', 'label_col': 'class_numeric', 'class_list': 'ten-classes', 'batch_size': 1024, 'input_dim': 1}
	{'arch': 'BERT', 'device': device(type='mps'), 'loadable': 'prajjwal1/bert-tiny', 'optimizer': <class 'torch.optim.adamw.AdamW'>, 'lr': 2e-05, 'loss_fn': <class 'torch.nn.modules.loss.CrossEntropyLoss'>, 'num_epochs': 2}
Directory /Users/diego/Desktop/iteso/TOG/bin/gen/TRAIN/ten-classes/BERT/Spv/prajjwal1/bert-tiny does not exist. Creating it and continuing withexecution
(1051, 12) {'PH2': True, 'PH3': True, 'reducer': 'kpca', 'kernel': 'poly', 'n': 1, 'data_unit': 'Spv', 'label_col': 'class_numeric', 'class_list': 'ten-classes', 'batch_size': 1024, 'input_dim': 1}
(1051, 12) {'P

# SCORE ANALYSIS

In [8]:
import pandas as pd
from datetime import datetime

## KNN

In [9]:
knn_scores_df = pd.DataFrame(sup.knn_score_tracker, columns=sup.knn_scores_columns)

In [10]:
knn_scores_df

,class_list,accuracy,data_unit,PH2,PH3,reducer,kernel,n,k
0,ten-classes,0.931554,AHpf,True,True,pca,,1,1
1,ten-classes,0.925733,AHpf,True,True,pca,,1,2
2,ten-classes,0.950422,AHpf,True,True,pca,,1,3
3,ten-classes,0.945203,AHpf,True,True,pca,,1,4
4,ten-classes,0.952027,AHpf,True,True,pca,,1,5
...,...,...,...,...,...,...,...,...,...
34777,ten-classes,0.440758,Spv,False,False,,,-1,27
34778,ten-classes,0.436019,Spv,False,False,,,-1,28
34779,ten-classes,0.407583,Spv,False,False,,,-1,29
34780,ten-classes,0.417062,Spv,False,False,,,-1,30


In [11]:
knn_scores_df.sort_values(by="accuracy", ascending=False).head(50)

,class_list,accuracy,data_unit,PH2,PH3,reducer,kernel,n,k
11441,ten-classes,0.994380,AHpf,False,True,kpca,rbf,31,3
11255,ten-classes,0.994179,AHpf,False,True,kpca,rbf,30,3
10697,ten-classes,0.994179,AHpf,False,True,kpca,rbf,27,3
10666,ten-classes,0.994179,AHpf,False,True,kpca,poly,27,3
10852,ten-classes,0.994179,AHpf,False,True,kpca,poly,28,3
10883,ten-classes,0.994179,AHpf,False,True,kpca,rbf,28,3
10139,ten-classes,0.993978,AHpf,False,True,kpca,rbf,24,3
10480,ten-classes,0.993978,AHpf,False,True,kpca,poly,26,3
10325,ten-classes,0.993978,AHpf,False,True,kpca,rbf,25,3
10294,ten-classes,0.993978,AHpf,False,True,kpca,poly,25,3


In [12]:
now = datetime.now().strftime("%Y-%m-%d_%H:%M:%S")
sup.create_dir_if_not_exists(os.path.join(sup.TRAIN_SCORES_ROOT, "two-classes", sup.TRAIN_KNN_CODE))
knn_scores_df.to_csv(os.path.join(sup.TRAIN_SCORES_ROOT,"two-classes", sup.TRAIN_KNN_CODE, f"{now}.csv"), index=False)

Directory /Users/diego/Desktop/iteso/TOG/scores/two-classes/KNN exists. Continuing with execution


## BERT

In [13]:
bert_scores_df = pd.DataFrame(sup.bert_score_tracker, columns=sup.bert_scores_columns)

In [14]:
bert_scores_df

,class_list,accuracy,data_unit,PH2,PH3,reducer,kernel,n,loadble,lr,optimizer,loss_fn,num_epochs
0,ten-classes,0.104265,Spv,True,True,pca,,1,prajjwal1/bert-tiny,0.00002,AdamW (\nParameter Group 0\n amsgrad: False...,CrossEntropyLoss(),2
1,ten-classes,0.104265,Spv,True,True,kpca,poly,1,prajjwal1/bert-tiny,0.00002,AdamW (\nParameter Group 0\n amsgrad: False...,CrossEntropyLoss(),2
2,ten-classes,0.104265,Spv,True,True,kpca,rbf,1,prajjwal1/bert-tiny,0.00002,AdamW (\nParameter Group 0\n amsgrad: False...,CrossEntropyLoss(),2
3,ten-classes,0.104265,Spv,True,True,kpca,sigmoid,1,prajjwal1/bert-tiny,0.00002,AdamW (\nParameter Group 0\n amsgrad: False...,CrossEntropyLoss(),2
4,ten-classes,0.104265,Spv,True,True,kpca,cosine,1,prajjwal1/bert-tiny,0.00002,AdamW (\nParameter Group 0\n amsgrad: False...,CrossEntropyLoss(),2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
369,ten-classes,0.094787,Spv,False,True,kpca,rbf,31,prajjwal1/bert-tiny,0.00002,AdamW (\nParameter Group 0\n amsgrad: False...,CrossEntropyLoss(),2
370,ten-classes,0.000000,Spv,False,True,kpca,sigmoid,31,prajjwal1/bert-tiny,0.00002,AdamW (\nParameter Group 0\n amsgrad: False...,CrossEntropyLoss(),2
371,ten-classes,0.000000,Spv,False,True,kpca,cosine,31,prajjwal1/bert-tiny,0.00002,AdamW (\nParameter Group 0\n amsgrad: False...,CrossEntropyLoss(),2
372,ten-classes,0.000000,Spv,False,True,umap,,31,prajjwal1/bert-tiny,0.00002,AdamW (\nParameter Group 0\n amsgrad: False...,CrossEntropyLoss(),2


In [15]:
bert_scores_df.sort_values(by="accuracy", ascending=False).head(50)

,class_list,accuracy,data_unit,PH2,PH3,reducer,kernel,n,loadble,lr,optimizer,loss_fn,num_epochs
36,ten-classes,0.151659,Spv,True,True,pca,,7,prajjwal1/bert-tiny,0.00002,AdamW (\nParameter Group 0\n amsgrad: False...,CrossEntropyLoss(),2
234,ten-classes,0.142180,Spv,False,True,umap,,8,prajjwal1/bert-tiny,0.00002,AdamW (\nParameter Group 0\n amsgrad: False...,CrossEntropyLoss(),2
12,ten-classes,0.132701,Spv,True,True,pca,,3,prajjwal1/bert-tiny,0.00002,AdamW (\nParameter Group 0\n amsgrad: False...,CrossEntropyLoss(),2
205,ten-classes,0.127962,Spv,False,True,pca,,4,prajjwal1/bert-tiny,0.00002,AdamW (\nParameter Group 0\n amsgrad: False...,CrossEntropyLoss(),2
213,ten-classes,0.127962,Spv,False,True,kpca,rbf,5,prajjwal1/bert-tiny,0.00002,AdamW (\nParameter Group 0\n amsgrad: False...,CrossEntropyLoss(),2
136,ten-classes,0.127962,Spv,True,True,kpca,cosine,23,prajjwal1/bert-tiny,0.00002,AdamW (\nParameter Group 0\n amsgrad: False...,CrossEntropyLoss(),2
273,ten-classes,0.123223,Spv,False,True,kpca,rbf,15,prajjwal1/bert-tiny,0.00002,AdamW (\nParameter Group 0\n amsgrad: False...,CrossEntropyLoss(),2
73,ten-classes,0.118483,Spv,True,True,kpca,poly,13,prajjwal1/bert-tiny,0.00002,AdamW (\nParameter Group 0\n amsgrad: False...,CrossEntropyLoss(),2
193,ten-classes,0.118483,Spv,False,True,pca,,2,prajjwal1/bert-tiny,0.00002,AdamW (\nParameter Group 0\n amsgrad: False...,CrossEntropyLoss(),2
247,ten-classes,0.118483,Spv,False,True,pca,,11,prajjwal1/bert-tiny,0.00002,AdamW (\nParameter Group 0\n amsgrad: False...,CrossEntropyLoss(),2


In [16]:
now = datetime.now().strftime("%Y-%m-%d_%H:%M:%S")
sup.create_dir_if_not_exists(os.path.join(sup.TRAIN_SCORES_ROOT, "two-classes", sup.TRAIN_BERT_CODE))
bert_scores_df.to_csv(os.path.join(sup.TRAIN_SCORES_ROOT, "two-classes", sup.TRAIN_BERT_CODE, f"{now}.csv"), index=False)

Directory /Users/diego/Desktop/iteso/TOG/scores/two-classes/BERT does not exist. Creating it and continuing withexecution
